# Hackathon

Some utilities

## Import Utils

In [ ]:
!pip install keras
!pip install tables

In [1]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/balanced_7classes.h5'
PATH_PREDICT_WITHOUT_GT = '/notebooks/data/pred_students/pred_from_half/pred_eighties_from_half_1_without_gt.h5'
PATH_SUBMIT = '/notebooks/ChallengeHacka/pred_eighties_from_half_1_gr6.csv'
# PATH_PREDICT_WITH_GT = '/pred_teachers/pred_eighties_from_half_1.h5'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
BATCH_SIZE = 32
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout
import keras.layers.normalization 
from keras.callbacks import Callback
%load_ext autoreload
%autoreload 2

In [3]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [4]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idx = idxs[b*batch_size]
            X = f['S2'][batch_idx:batch_idx+batch_size, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idx:batch_idx+batch_size, :]
            for i in range(len(Y)) :
                a = Y[i]
                if a < 4:
                    Y[i] -= 1
                elif a == 5:
                    Y[i] = 3
                elif a == 10:
                    Y[i] = 4
                elif a == 12:
                    Y[i] = 5
                elif a == 19:
                    Y[i] = 6
                else : 
                    print('value class out of range : %d ' %a)
                            
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 7)

In [5]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)
train_idxs, val_idxs = split_train_val(shuffled_idxs, 0.2)

print(len(idxs))

2100000


In [6]:
train_gen = generator(PATH_DATA, BATCH_SIZE, train_idxs)
train_batch_count = get_batch_count(train_idxs, BATCH_SIZE)

val_gen = generator(PATH_DATA, BATCH_SIZE, val_idxs)
val_batch_count = get_batch_count(val_idxs, BATCH_SIZE)

In [7]:
print(train_batch_count, val_batch_count)

52500 13125


In [8]:
label = h5.File(PATH_DATA)['TOP_LANDCOVER']
images = h5.File(PATH_DATA)['S2']

In [10]:
unique, counts = np.unique(label, return_counts=True)
# panda_prediction.describe()
print(unique)
print(counts)

[ 1.  2.  3.  5. 10. 12. 19.]
[300000 300000 300000 300000 300000 300000 300000]


In [11]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

# Instanciation du model

In [12]:
input_shape = (16,16,4)

dropout_rate=0.5

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Fit

In [ ]:
# optim = keras.optimizers.Adam(lr=0.001)
optim = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy', top_3_accuracy ])

history = model.fit_generator(train_gen, steps_per_epoch=train_batch_count, epochs=3, verbose=1, validation_data=val_gen, nb_val_samples=1000)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=52500, verbose=1, validation_data=<generator..., validation_steps=1000, epochs=3)`
  


Epoch 1/3
52500/52500 [==============================] - 593s 11ms/step - loss: 1.2453 - acc: 0.5263 - top_3_accuracy: 0.8613 - val_loss: 2.8980 - val_acc: 0.2201 - val_top_3_accuracy: 0.5522
Epoch 2/3
52500/52500 [==============================] - 811s 15ms/step - loss: 1.2446 - acc: 0.5270 - top_3_accuracy: 0.8635 - val_loss: 2.7451 - val_acc: 0.1846 - val_top_3_accuracy: 0.4894
Epoch 3/3
35443/52500 [===================>..........] - ETA: 5:21 - loss: 1.2511 - acc: 0.5256 - top_3_accuracy: 0.8620

## Prediction routines

In order to submit a result here are some gits

In [15]:
import os 
def prediction_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        X = f['S2'][batch_idxs, :,:,:]
        yield np.array(X)

In [38]:
pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
print(len(pred_idx))
pred_gen = prediction_generator(PATH_PREDICT_WITHOUT_GT, BATCH_SIZE, pred_idx)
prediction = model.predict_generator(pred_gen, steps=get_batch_count(pred_idx, BATCH_SIZE), verbose=1)
print(len(prediction))

preds = np.argmax(prediction, axis = 1)

unique, counts = np.unique(preds, return_counts=True)
print(unique)
print(counts)

for i, p in enumerate(preds):
    if p == 0 :
        preds[i] = 1
    elif p == 1:
        preds[i] =  2
    elif p == 2:
        preds[i] =  3
    elif p == 3:
        preds[i] =  5
    elif p == 4:
        preds[i] =  10
    elif p == 5:
        preds[i] =  12
    elif p == 6:
        preds[i] =  19

panda_prediction = pd.DataFrame(preds)

panda_prediction

panda_prediction.columns = ['TOP_LANDCOVER']


241700
7554/7554 [==============================] - 25s 3ms/step
241700
[0 1 2 3 4 5 6]
[ 34508 134993   5897   1418  16722  33700  14462]


In [33]:
x = np.array([1, 2, 8, 10, 3, 2])

y = np.where(x == 2)[0]
y

array([1, 5])

In [ ]:
unique, counts = np.unique(preds, return_counts=True)
# panda_prediction.describe()
print(unique)
print(counts)

In [39]:
unique, counts = np.unique(preds, return_counts=True)
# panda_prediction.describe()
print(unique)
print(counts)

[ 1  2  3  5 10 12 19]
[ 34508 134993   5897   1418  16722  33700  14462]


In [40]:
panda_prediction.to_csv(PATH_SUBMIT, index_label = 'ID')